In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import resnet50
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the TF-Flowers Dataset
# ---------------------------------------------------
# Create train/validation split
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in TF-Flowers
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for ResNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_resnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = resnet50.preprocess_input(image)  # Use resnet50's preprocess_input
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_resnet = (dataset_train
                   .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                   .batch(batch_size)
                   .prefetch(AUTOTUNE))

test_ds_resnet = (dataset_test
                  .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                  .batch(batch_size)
                  .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define ResNet50 Model
# ---------------------------------------------------
def create_resnet(input_shape, num_classes):
    base_model = resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="ResNet50")
    return model

# ---------------------------------------------------
# 4. Compile and Train the Model
# ---------------------------------------------------
# ResNet50 Model
resnet_model = create_resnet((IMG_SIZE, IMG_SIZE, 3), num_classes)
resnet_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning ResNet50 on TF-Flowers ---")
history_resnet = resnet_model.fit(train_ds_resnet, epochs=50, validation_data=test_ds_resnet, verbose=1)

# Evaluate ResNet50
loss_resnet, acc_resnet = resnet_model.evaluate(test_ds_resnet, verbose=0)
print(f"ResNet50 Final Accuracy: {acc_resnet:.4f}")

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- Fine-Tuning ResNet50 on TF-Flowers ---
Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 50s 304ms/step - accuracy: 0.2142 - loss: 2.1428 - val_accuracy: 0.5218 - val_loss: 1.2064
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step - accuracy: 0.5028 - loss: 1.2777 - val_accuracy: 0.7180 - val_loss: 0.8089
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 117ms/step - accuracy: 0.6677 - loss: 0.9162 - val_accuracy: 0.7752 - val_loss: 0.6418
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 112ms/step - accuracy: 0.7316 - loss: 0.7401 - val_accuracy: 0.8215 - val_loss: 0.5452
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 122ms/step - accuracy: 0.7950 - loss: 0.6067 - val_accuracy: 0.8392 - val_loss: 0.4858
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.8143 - loss: 0.5516 - val_accuracy: 0.8610 - val_loss: 0.4457
Ep